# Training and Evaluation for CINIC10 using SimpleCNN and OptimalCNN

This notebook loads the converted dataset, trains the SimpleCNN model, and performs evaluation and inference on the validation set.

In [1]:
import os
import torch
import torch.optim as optim
import torch.nn as nn

from DataObjects import DataLoader
from Architectures.SimpleCNN import SimpleCNN
from Architectures.OptimalCNN import OptimalCNN
from Architectures.StochasticDepthCNN import StochasticDepthCNN

from typing import Optional, Tuple

from utils import save_model, load_model

In [2]:
def train_model(model: nn.Module, train_loader: DataLoader, val_loader: DataLoader,
                num_epochs: int = 10, lr: float = 0.001,
                device: torch.device = None) -> None:
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion: nn.Module = nn.CrossEntropyLoss()
    optimizer: torch.optim.Optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(num_epochs):
        model.train()
        train_loss: float = 0.0
        train_correct: int = 0
        total_train: int = 0
        
        for batch in train_loader:
            inputs = batch.data.to(device)
            labels = batch.labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels).item()
            total_train += labels.size(0)
        
        avg_train_loss = train_loss / total_train
        train_acc = train_correct / total_train
        
        model.eval()
        val_loss: float = 0.0
        val_correct: int = 0
        total_val: int = 0
        
        with torch.no_grad():
            for batch in val_loader:
                inputs = batch.data.to(device)
                labels = batch.labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels).item()
                total_val += labels.size(0)
        
        avg_val_loss = val_loss / total_val
        val_acc = val_correct / total_val
        
        print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {avg_train_loss:.4f}, Train acc: {train_acc:.4f} | Val loss: {avg_val_loss:.4f}, Val acc: {val_acc:.4f}")

def infer(model: nn.Module, data_loader: DataLoader,
          device: torch.device = None) -> list:
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    predictions: list = []
    
    with torch.no_grad():
        for batch in data_loader:
            inputs = batch.data.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().tolist())
    
    return predictions


def evaluate(model: nn.Module,
             test_loader: Optional[DataLoader] = None,
             device: Optional[torch.device] = None) -> Tuple[float, float]:

    if test_loader is None:
        test_dir = os.path.join("Data", "Data_converted", "test")
        test_loader = DataLoader(test_dir, batch_size=64, shuffle=True)
        
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    model.eval()
    
    test_loss = 0.0
    test_correct = 0
    total_test = 0
    
    with torch.no_grad():
        for batch in test_loader:
            inputs = batch.data.to(device)
            labels = batch.labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            test_correct += torch.sum(preds == labels).item()
            total_test += labels.size(0)
    
    avg_test_loss = test_loss / total_test
    test_acc = test_correct / total_test
    
    print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_acc:.4f}")
    return avg_test_loss, test_acc


In [3]:
# Setup directories and DataLoaders
train_dir = os.path.join("Data", "Data_converted", "train")
val_dir = os.path.join("Data", "Data_converted", "valid")

train_loader = DataLoader(train_dir, batch_size=64, shuffle=True, max_per_class=150)
val_loader = DataLoader(val_dir, batch_size=64, shuffle=False, max_per_class=150) 

# Instantiate the model
model = SimpleCNN(num_classes=10)

In [4]:
# Train the model
train_model(model, train_loader, val_loader, num_epochs=10, lr=0.001)

Epoch 1/10 - Train loss: 6.6857, Train acc: 0.1033 | Val loss: 2.3000, Val acc: 0.1080
Epoch 2/10 - Train loss: 2.2259, Train acc: 0.1673 | Val loss: 2.1438, Val acc: 0.2213
Epoch 3/10 - Train loss: 1.9777, Train acc: 0.3020 | Val loss: 2.0973, Val acc: 0.2513
Epoch 4/10 - Train loss: 1.6652, Train acc: 0.4280 | Val loss: 2.1132, Val acc: 0.2747
Epoch 5/10 - Train loss: 1.4246, Train acc: 0.5160 | Val loss: 2.2301, Val acc: 0.2840
Epoch 6/10 - Train loss: 1.1182, Train acc: 0.6227 | Val loss: 2.3942, Val acc: 0.3127
Epoch 7/10 - Train loss: 0.8696, Train acc: 0.7127 | Val loss: 2.4653, Val acc: 0.3133
Epoch 8/10 - Train loss: 0.6512, Train acc: 0.7840 | Val loss: 2.7072, Val acc: 0.3087
Epoch 9/10 - Train loss: 0.4566, Train acc: 0.8667 | Val loss: 3.0975, Val acc: 0.3307
Epoch 10/10 - Train loss: 0.3268, Train acc: 0.8920 | Val loss: 3.5846, Val acc: 0.3347


In [5]:
test_dir = os.path.join("Data", "Data_converted", "test")
test_loader = DataLoader(test_dir, batch_size=64, shuffle=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()

model.eval()
test_loss = 0.0
test_correct = 0
total_test = 0

with torch.no_grad():
    for batch in test_loader:
        inputs = batch.data.to(device)
        labels = batch.labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        test_correct += torch.sum(preds == labels).item()
        total_test += labels.size(0)

avg_test_loss = test_loss / total_test
test_acc = test_correct / total_test

print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


Test Loss: 4.4494, Test Accuracy: 0.2675


In [6]:
# Full training
train_loader_full = DataLoader(train_dir, batch_size=64, shuffle=True)
val_loader_full = DataLoader(val_dir, batch_size=64, shuffle=False) 

# Instantiate the model
model_2 = SimpleCNN(num_classes=10)

In [7]:
# Train the model
train_model(model_2, train_loader, val_loader, num_epochs=10, lr=0.001)

Epoch 1/10 - Train loss: 10.2835, Train acc: 0.1120 | Val loss: 2.2460, Val acc: 0.1753
Epoch 2/10 - Train loss: 2.0958, Train acc: 0.2527 | Val loss: 2.0453, Val acc: 0.2393
Epoch 3/10 - Train loss: 1.8051, Train acc: 0.3533 | Val loss: 1.9665, Val acc: 0.2880
Epoch 4/10 - Train loss: 1.5452, Train acc: 0.4680 | Val loss: 1.9917, Val acc: 0.2867
Epoch 5/10 - Train loss: 1.3262, Train acc: 0.5400 | Val loss: 2.0860, Val acc: 0.3040
Epoch 6/10 - Train loss: 1.0859, Train acc: 0.6340 | Val loss: 2.2698, Val acc: 0.3220
Epoch 7/10 - Train loss: 0.9261, Train acc: 0.6867 | Val loss: 2.3528, Val acc: 0.3187
Epoch 8/10 - Train loss: 0.7389, Train acc: 0.7660 | Val loss: 2.5201, Val acc: 0.3380
Epoch 9/10 - Train loss: 0.6293, Train acc: 0.7967 | Val loss: 2.7999, Val acc: 0.3373
Epoch 10/10 - Train loss: 0.4693, Train acc: 0.8620 | Val loss: 2.8354, Val acc: 0.3347


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_2.to(device)

criterion = nn.CrossEntropyLoss()

model_2.eval()
test_loss = 0.0
test_correct = 0
total_test = 0

with torch.no_grad():
    for batch in test_loader:
        inputs = batch.data.to(device)
        labels = batch.labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        test_correct += torch.sum(preds == labels).item()
        total_test += labels.size(0)

avg_test_loss = test_loss / total_test
test_acc = test_correct / total_test

print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


Test Loss: 4.4494, Test Accuracy: 0.2675


In [9]:
## Different architecture
model_Optimal = OptimalCNN(num_classes=10)
model_Stochastic = StochasticDepthCNN(num_classes=10)

In [10]:
# Train the model
# On full set, it trains for really long time, reserve around 6 hours or reduce the size of the training set
train_model(model_Optimal, train_loader_full, val_loader_full, num_epochs=20, lr=0.005)

Epoch 1/20 - Train loss: 1.4979, Train acc: 0.4528 | Val loss: 1.3284, Val acc: 0.5117
Epoch 2/20 - Train loss: 1.2171, Train acc: 0.5647 | Val loss: 1.1851, Val acc: 0.5766
Epoch 3/20 - Train loss: 1.0974, Train acc: 0.6120 | Val loss: 1.0521, Val acc: 0.6255
Epoch 4/20 - Train loss: 1.0052, Train acc: 0.6463 | Val loss: 1.0027, Val acc: 0.6413
Epoch 5/20 - Train loss: 0.9302, Train acc: 0.6721 | Val loss: 1.0255, Val acc: 0.6427
Epoch 6/20 - Train loss: 0.8679, Train acc: 0.6934 | Val loss: 0.9478, Val acc: 0.6678
Epoch 7/20 - Train loss: 0.8046, Train acc: 0.7162 | Val loss: 0.9418, Val acc: 0.6706
Epoch 8/20 - Train loss: 0.7543, Train acc: 0.7333 | Val loss: 0.9796, Val acc: 0.6721
Epoch 9/20 - Train loss: 0.6992, Train acc: 0.7535 | Val loss: 0.8934, Val acc: 0.6893
Epoch 10/20 - Train loss: 0.6532, Train acc: 0.7677 | Val loss: 0.9709, Val acc: 0.6800
Epoch 11/20 - Train loss: 0.6048, Train acc: 0.7846 | Val loss: 0.9759, Val acc: 0.6817
Epoch 12/20 - Train loss: 0.5645, Train a

In [11]:
# train_model(model_Stochastic, train_loader_full, val_loader_full, num_epochs=10, lr=0.001)

In [12]:
# Evaluate
evaluate(model_Optimal, test_loader)

Test Loss: 1.2586, Test Accuracy: 0.6792


(1.2585868682649402, 0.6792111111111111)

In [13]:
evaluate(model_Stochastic, test_loader)

Test Loss: 2.3101, Test Accuracy: 0.0899


(2.3100872038947213, 0.08987777777777778)

In [14]:
# Saving model into the pytorch format
save_model(model_Optimal, "Models_Pytorch_saved/OptimalCNN_trained_saved.pth")

Model saved successfully at Models_Pytorch_saved/OptimalCNN_trained_saved.pth


In [15]:
# Now load it
model_Optimal_loaded = load_model("Models_Pytorch_saved/OptimalCNN_trained_saved.pth")

Model loaded successfully from Models_Pytorch_saved/OptimalCNN_trained_saved.pth


In [16]:
# Check if evaluated the same with the loaded model
evaluate(model_Optimal_loaded, test_loader)
# yes!

Test Loss: 1.2586, Test Accuracy: 0.6792


(1.2585868682649402, 0.6792111111111111)